# Movilidad Sustentable
## Ciencia de Datos e Inteligencia de Negocios


### Configuración

In [1]:
"""
Bicicletas: Recorridos
Clima: PRCP TAVG TEMP[C] HUM[%] PNM[hPa] DD[gr] FF[km/hr]                                                 
2020-2021
Data acquisition & wrangling
"""

import os
import sys
import urllib.request

import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import datetime
from datetime import date, timedelta
import re

datadir = 'datasets/'

In [ ]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    from google.colab import drive
    drive.mount(
      '/content/drive/', 
    )
    if drive:
        datadir = '/content/drive/MyDrive/Academicos/UTN/BI/datasets/'
else:
    print('Not running on CoLab')

# Adquisición


In [2]:
insumos = {
  "recorridos-2020" : {
    "remote" : 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/transporte/bicicletas-publicas/recorridos-realizados-2020.zip',
    "local" : datadir + 'recorridos-realizados-2020.zip',
    "df" : pd.DataFrame(),
  },
  "recorridos-2021" : {
    "remote" : 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/transporte/bicicletas-publicas/recorridos-realizados-2021.zip',   
    "local" : datadir + 'recorridos-realizados-2021.zip',
    "df" : pd.DataFrame(),
  },
  "clima" : {
    "remote" : '',   
    "local" : datadir + 'weather-data-ba.csv',
    "df" : pd.DataFrame(),
  },
#   "estaciones" : {
#     "remote" : 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/transporte/estaciones-bicicletas-publicas/nuevas-estaciones-bicicletas-publicas.xlsx',   
#     "local" : datadir + 'nuevas_estaciones.xlsx',
#     "df" : pd.DataFrame(),
#   },
  "feriados" : {
    "remote" : 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/transporte/estaciones-bicicletas-publicas/nuevas-estaciones-bicicletas-publicas.xlsx',   
    "local" : datadir + 'dias_festivos.csv',
    "df" : pd.DataFrame(),
  },
#   "rutas" : {
#     "remote" : '',   
#     "local" : datadir + 'rutas.csv',
#     "df" : pd.DataFrame(),
#   },
}

In [3]:
"""
Cargar insumos
"""
def cargar_insumos( insumos ):
    for i in insumos:
        if not os.path.exists( insumos[i]["local"] ):
            print("Descargando:", insumos[i]["remote"])
            urllib.request.urlretrieve(
              insumos[i]["remote"],
              insumos[i]["local"]
            )
        if "zip" in insumos[i]["local"]:
            insumos[i]["df"] = pd.read_csv(
                insumos[i]["local"],
                compression = 'zip',
                encoding = 'utf-8', 
            )
        if "csv" in insumos[i]["local"]:
            insumos[i]["df"] = pd.read_csv(
                insumos[i]["local"],
            )
        if "xlsx" in insumos[i]["local"]:
            insumos[i]["df"] = pd.read_excel(
                insumos[i]["local"],
            )
        print( 'Cargado: ', insumos[i]["local"] )
    return insumos

In [4]:
insumos = cargar_insumos( insumos )

Cargado:  datasets/recorridos-realizados-2020.zip
Cargado:  datasets/recorridos-realizados-2021.zip
Cargado:  datasets/weather-data-ba.csv
Cargado:  datasets/dias_festivos.csv


# Subproductos

In [5]:
derivados = {
  "recorridos" : {
    "df"     : pd.DataFrame(),
    "local"  : datadir + 'recorridos_clima-2020-2021.csv',
    "remote" : '',   
  },
#   "vueltas" : {
#     "df"     : pd.DataFrame(),
#     "local"  : datadir + 'vueltas.csv',
#     "remote" : '',   
#   },
  "estaciones" : {
    "df"     : pd.DataFrame(),
    "local"  : datadir + 'estaciones.csv',
    "remote" : '',   
  },
  "distancias" : {
    "df"     : pd.DataFrame(),
    "local"  : datadir + 'distancias.csv',
    "remote" : '',   
  },
  "rutas" : {
    "df"     : pd.DataFrame(),
    "local"  : datadir + 'rutas.csv',
    "remote" : '',   
  },
  "clima" : {
    "df"     : pd.DataFrame(),
    "local"  : datadir + 'clima_datohorario_2020_2021.csv', 
    "remote" : '',
  },
  "pronostico" : {
    "df"     : pd.DataFrame(),
    "local"  : datadir + 'pronostico_2020_2021.csv', 
    "remote" : '',
  },
}

 ## Normalizacion

In [6]:
"""
Renombrar y filtrar columnas 2021
"""
encabezados2021 = {
    'ID'                                  : 'id',
    'Duración'                            : 'duracion_recorrido',
    'Estado cerrado'                      : 'estado_cerrado',
    'ID de ciclista'                      : 'id_usuario',
    'Tipo de ciclista'                    : 'cilicsta_tipo',
    'Modelo de bicicleta'                 : 'bicicleta_modelo',   
    'Id de estación de inicio'            : 'id_estacion_origen', 
    'Nombre de estación de inicio'        : 'nombre_estacion_origen',
    'Fecha de inicio'                     : 'fecha_origen_recorrido',     
    'Origen de viaje'                     : 'direccion_estacion_origen',  
    'lon_estacion_origen'                 : 'lon_estacion_origen',        
    'lat_estacion_origen'                 : 'lat_estacion_origen',
    'Id de estación de fin de viaje'      : 'id_estacion_destino',     
    'Nombre de estación de fin de viaje'  : 'nombre_estacion_destino',
    'Fecha de fin'                        : 'fecha_destino_recorrido',          
#     'direccion_estacion_destino'          : 'direccion_estacion_origen',           
#     'lon_estacion_destino'                : 'lon_estacion_destino',       
#     'lat_estacion_destino'                : 'lat_estacion_destino',
#     'periodo'                             : 'periodo',  
}
insumos["recorridos-2021"]["df"] = insumos["recorridos-2021"]["df"].rename( 
    columns = encabezados2021
)
insumos["recorridos-2021"]["df"]["id"] = insumos["recorridos-2021"]["df"]['id'].astype('Int64')

In [7]:
"""
Concatenar recorridos 2020:2021
"""
recorridos = [
    insumos["recorridos-2020"]["df"],
    insumos["recorridos-2021"]["df"]
]
derivados["recorridos"]["df"] = pd.concat( recorridos )
del recorridos  

In [8]:
encabezados = {
#     'id'                         : 'id',
    'duracion_recorrido'         : 'duracion',
#     'estado_cerrado'             : 'estado_cerrado',
    'id_usuario'                 : 'usuario_id',
#     'cilicsta_tipo'            : 'cilicsta_tipo',
#     'bicicleta_modelo'         : 'bicicleta_modelo',   
    'id_estacion_origen'         : 'origen_id', 
    'nombre_estacion_origen'     : 'origen_nombre',
    'fecha_origen_recorrido'     : 'origen_fecha',     
    'direccion_estacion_origen'  : 'origen_direccion',
    'lat_estacion_origen'        : 'origen_lat',  
    'long_estacion_origen'       : 'origen_lon',        
    'id_estacion_destino'        : 'destino_id',     
    'nombre_estacion_destino'    : 'destino_nombre',
    'fecha_destino_recorrido'    : 'destino_fecha',          
#     'direccion_estacion_destino' : 'destino_direccion', 
#     'lat_estacion_destino'       : 'destino_lat',          
#     'long_estacion_destino'      : 'destino_lon',       
#     'periodo'                    : 'periodo',  
}
derivados["recorridos"]["df"] = derivados["recorridos"]["df"].rename( columns = encabezados )
derivados["recorridos"]["df"] = derivados["recorridos"]["df"][ list( encabezados.values() ) ]

In [ ]:
# len(derivados["recorridos"]["df"]["destino_id"].unique())
# # # hay 3 destinos mas q viajes  
# # [nan, 288.0, 427.0, 47.0, 447.0]
# list(
#     set( derivados['recorridos']['df']['destino_nombre'].unique().tolist() ) -
#     set( derivados['recorridos']['df']['origen_nombre'].unique().tolist() )
# )
# #     3 destinos mas q viajes  [333, 397, 47]
# # [nan,
# #  'Balboa Definitivo',
# #  '333 - MIRO',
# #  '047 - Colegio Nacional de Buenos Aires',
# #  '397 - PLAZA MATHEU']

In [9]:
derivados["recorridos"]["df"].dropna( subset = [ "destino_id" ], inplace = True )
derivados["recorridos"]["df"] = derivados["recorridos"]["df"][
    derivados["recorridos"]["df"]['destino_nombre'] != 'Balboa Definitivo'
]

In [10]:
"""
Separar las columnas nombre de estacion de origen y nombre de estacion de destino 
Tienen la info correcta del id de las estaciones.
Divido la columna Nombre de estacion de fin de viaje porque contiene dos campos 
el Id y el nombre de la estacion de inicio de viaje.
"""
derivados["recorridos"]["df"][
    ['origen_id','origen_nombre']
] = derivados["recorridos"]["df"]['origen_nombre'].str.split('-', expand = True )
derivados["recorridos"]["df"]['origen_id'] = derivados["recorridos"]["df"]['origen_id'].str.strip(' ')
derivados["recorridos"]["df"]['origen_nombre'] = derivados["recorridos"]["df"]['origen_nombre'].str.strip(' ')
derivados["recorridos"]["df"][
    ['destino_id','destino_nombre']
] = derivados["recorridos"]["df"]['destino_nombre'].str.split('-', expand = True )
derivados["recorridos"]["df"]['destino_id'] = derivados["recorridos"]["df"]['destino_id'].str.strip(' ')
derivados["recorridos"]["df"]['destino_nombre'] = derivados["recorridos"]["df"]['destino_nombre'].str.strip(' ')

# # hay 3 destinos mas q viajes  [333, 397, 47]
# list(
#     set( derivados['recorridos']['df']['destino_id'].unique().tolist() )-
#     set( derivados['recorridos']['df']['origen_id'].unique().tolist() )
# )

In [11]:
derivados["recorridos"]["df"]['origen_id'] = derivados["recorridos"]["df"]["origen_id"].str.lstrip('0').astype(int)
derivados["recorridos"]["df"]['destino_id'] = derivados["recorridos"]["df"]["destino_id"].str.lstrip('0').astype(int)
derivados["recorridos"]["df"]["origen_nombre"]  = derivados["recorridos"]["df"]["origen_nombre"].str.upper()
derivados["recorridos"]["df"]["destino_nombre"]  = derivados["recorridos"]["df"]["destino_nombre"].str.upper()
derivados["recorridos"]["df"]["origen_direccion"]  = derivados["recorridos"]["df"]["origen_direccion"].str.upper()
# derivados["recorridos"]["df"]["destino_direccion"]  = derivados["recorridos"]["df"]["destino_direccion"].str.upper()

In [12]:
derivados["recorridos"]["df"].isnull().sum()

duracion                  0
usuario_id                0
origen_id                 0
origen_nombre             0
origen_fecha              0
origen_direccion          0
origen_lat          1328687
origen_lon          1328687
destino_id                0
destino_nombre            0
destino_fecha             0
dtype: int64

In [ ]:
# LIMPIAR Y PREPARAR ESTACIONES 
# derivados["estaciones"]["df"] = pd.DataFrame()

derivados["estaciones"]["df"]['id'] = insumos["estaciones"]["df"]['codigo']
derivados["estaciones"]["df"]['nombre'] = insumos["estaciones"]["df"]['nombre'].str.replace(
    '\d+', '',
    regex=True
).str.replace('-', '', regex=True).str.strip(' ').str.upper()
derivados["estaciones"]["df"]['oid'] = insumos["estaciones"]["df"]['id']
derivados["estaciones"]["df"]['direccion'] = insumos["estaciones"]["df"]['ubicacion'].str.upper()
derivados["estaciones"]["df"][
    ['lon','lat']
] = insumos["estaciones"]["df"]['WKT'].str.replace('POINT ','').str.lstrip('(').str.rstrip(')').str.split(' ', expand = True )
derivados["estaciones"]["df"]['anclajes'] = insumos["estaciones"]["df"]['anclajes_t']
derivados["estaciones"]["df"] # clean

In [14]:
# AGREGAR DATA DE VIAJES 
salidas = derivados["recorridos"]["df"].groupby(
    'origen_id' 
).agg(
    salidas = pd.NamedAgg( column = "origen_id", aggfunc = "count"),
).reset_index()

arribos = derivados["recorridos"]["df"].groupby(
    'destino_id'
).agg(
  arribos = pd.NamedAgg( column = "destino_id", aggfunc = "count"),
).reset_index()

salidas_arribos = pd.merge(
    left     = salidas,
    right    = arribos,
    how      = 'left',
    left_on  = ['origen_id'],
    right_on = ['destino_id']
)

derivados["estaciones"]["df"] = pd.merge(
    left     = derivados["estaciones"]["df"],
    right    = salidas_arribos,
    how      = 'left',
    left_on  = ['id','id'],
    right_on = ['origen_id','destino_id']
)

del [salidas, arribos, salidas_arribos]

derivados["estaciones"]["df"]["salidas"] = derivados["estaciones"]["df"]["salidas"].fillna(0).astype(int)
derivados["estaciones"]["df"]["arribos"] = derivados["estaciones"]["df"]["arribos"].fillna(0).astype(int)

derivados["estaciones"]["df"] = derivados["estaciones"]["df"].drop(
    columns = ['origen_id','destino_id']
)
derivados["estaciones"]["df"] = derivados["estaciones"]["df"].sort_values(
   by = ['salidas'], ascending = [False]
)

# GRABAR CSV ESTACIONES
derivados["estaciones"]["df"].to_csv(
  derivados["estaciones"]["local"],
  index = False,
)


In [ ]:
# CARGAR ESTACIONES
# insumos["estaciones"]["df"] # raw
# derivados["estaciones"]["df"] = pd.read_csv(
#   derivados["estaciones"]["local"]
# )
# derivados["estaciones"]["df"]

In [ ]:
# print(
#     derivados["estaciones"]["df"]["salidas"].sum() -
#     derivados["estaciones"]["df"]["arribos"].sum()
# )

## RESAMPLING (REMUESTREO)

### Downsampling frequencies 

Tamaño de la muestra = Minuto

In [13]:
# https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.weekday.html
# https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.weekday.html
# https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.dayofweek.html#pandas.Series.dt.dayofweek

formato = '%Y%m%d %H:%M:%S'
derivados["recorridos"]["df"]['origen_datetime'] = pd.to_datetime(
    derivados["recorridos"]["df"]['origen_fecha'], 
    format = formato
)
derivados["recorridos"]["df"]['fecha'] = derivados["recorridos"]["df"]['origen_datetime'].dt.strftime('%Y-%m-%d')
# derivados["recorridos"]["df"]['anio'] = derivados["recorridos"]["df"]["fecha"].str.split('-')[0] 
derivados["recorridos"]["df"]['anio'] = derivados["recorridos"]["df"]['origen_datetime'].dt.strftime('%Y').astype(str).astype(int)
derivados["recorridos"]["df"]['ndia'] = derivados["recorridos"]["df"]['origen_datetime'].dt.strftime('%j').astype(str).astype(int)
# FECHA DELTA Y ESTACIONALIDAD SEMANAL
primer_anio = derivados["recorridos"]["df"]['anio'].min()
derivados["recorridos"]["df"].eval( 'ddia = ( ( anio % @primer_anio ) * 365 ) + ndia', inplace = True )
derivados["recorridos"]["df"]['wdia'] = derivados["recorridos"]["df"]['origen_datetime'].dt.strftime('%w').astype(str).astype(int)
# DIA HABIL 
derivados["recorridos"]["df"]["dhabil"] = 0
# Weekday as a decimal number, where 0 is Sunday and 6 is Saturday.
derivados["recorridos"]["df"]["dhabil"][ (derivados["recorridos"]["df"]['wdia'] >= 1) | (derivados["recorridos"]["df"]['wdia'] <= 5) ] = 1
derivados["recorridos"]["df"]['hora'] = derivados["recorridos"]["df"]['origen_datetime'].dt.strftime('%H').astype(str).astype(int)
derivados["recorridos"]["df"]['mins'] = derivados["recorridos"]["df"]['origen_datetime'].dt.strftime('%M').astype(str).astype(int)

# derivados["recorridos"]["df"].eval( 'dhabil = wdia > 4', inplace = True )
# derivados["recorridos"]["df"]['dhabil'] = derivados["recorridos"]["df"]['dhabil'].astype(int)

# derivados["recorridos"]["df"]['mes']  = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%b').str.lower()
# derivados["recorridos"]["df"]['nmes'] = derivados["recorridos"]["df"]['origen_datetime'].dt.strftime('%m').astype(str).astype(int)

In [14]:
# ESTACIONALIDAD TRIMESTRAL
derivados["recorridos"]["df"]["trimestre"] = pd.cut( 
    derivados["recorridos"]["df"].ndia,
    include_lowest = True,
    bins = 4,
    labels = False,
)
derivados["recorridos"]["df"]['trimestre'].unique()

array([3, 2, 1, 0])

In [15]:
derivados["recorridos"]["df"] = derivados["recorridos"]["df"].drop( columns = { 'anio','origen_datetime'} )
derivados["recorridos"]["df"].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3331632 entries, 0 to 1328725
Data columns (total 19 columns):
 #   Column            Dtype  
---  ------            -----  
 0   duracion          int64  
 1   usuario_id        int64  
 2   origen_id         int64  
 3   origen_nombre     object 
 4   origen_fecha      object 
 5   origen_direccion  object 
 6   origen_lat        float64
 7   origen_lon        float64
 8   destino_id        int64  
 9   destino_nombre    object 
 10  destino_fecha     object 
 11  fecha             object 
 12  ndia              int64  
 13  ddia              int64  
 14  wdia              int64  
 15  dhabil            int64  
 16  hora              int64  
 17  mins              int64  
 18  trimestre         int64  
dtypes: float64(2), int64(11), object(6)
memory usage: 508.4+ MB


In [17]:
derivados["recorridos"]["df"].isnull().sum()

duracion            0
usuario_id          0
origen_id           0
origen_nombre       0
origen_fecha        0
origen_direccion    0
destino_id          0
destino_nombre      0
destino_fecha       0
fecha               0
ndia                0
wdia                0
dhabil              0
hora                0
mins                0
ddia                0
trimestre           0
dtype: int64

In [20]:
derivados["recorridos"]["df"].describe()

,duracion,usuario_id,origen_id,destino_id,ndia,wdia,dhabil,hora,mins,ddia,trimestre
count,3.331632e+06,3.331632e+06,3.331632e+06,3.331632e+06,3.331632e+06,3.331632e+06,3.331632e+06,3.331632e+06,3.331632e+06,3.331632e+06,3.331632e+06
mean,1.360008e+03,4.366418e+05,1.526023e+02,1.537240e+02,1.727415e+02,3.032122e+00,2.768514e-01,1.431416e+01,2.915972e+01,3.183070e+02,1.394215e+00
std,5.219582e+03,2.613882e+05,1.036271e+02,1.039965e+02,1.189105e+02,1.916750e+00,4.474425e-01,5.293337e+00,1.726815e+01,1.274829e+02,1.247864e+00
min,0.000000e+00,3.700000e+01,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
25%,6.830000e+02,1.747990e+05,7.000000e+01,7.100000e+01,5.800000e+01,1.000000e+00,0.000000e+00,1.100000e+01,1.400000e+01,2.530000e+02,0.000000e+00
50%,1.095000e+03,4.971600e+05,1.420000e+02,1.420000e+02,1.510000e+02,3.000000e+00,0.000000e+00,1.500000e+01,2.900000e+01,3.320000e+02,1.000000e+00
75%,1.613000e+03,6.840670e+05,2.270000e+02,2.270000e+02,2.870000e+02,5.000000e+00,1.000000e+00,1.800000e+01,4.400000e+01,4.170000e+02,3.000000e+00
max,3.190094e+06,7.722650e+05,3.990000e+02,3.990000e+02,3.660000e+02,6.000000e+00,1.000000e+00,2.300000e+01,5.900000e+01,5.180000e+02,3.000000e+00


In [20]:
derivados["recorridos"]["df"][
    derivados["recorridos"]["df"]['duracion'] >= 86400 # 24hs * 60' * 60''
].info() # 360 VIAJES DE MAS DE 1 DIA

<class 'pandas.core.frame.DataFrame'>
Int64Index: 360 entries, 46714 to 1327256
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   duracion          360 non-null    int64 
 1   usuario_id        360 non-null    int64 
 2   origen_id         360 non-null    int64 
 3   origen_nombre     360 non-null    object
 4   origen_fecha      360 non-null    object
 5   origen_direccion  360 non-null    object
 6   destino_id        360 non-null    int64 
 7   destino_nombre    360 non-null    object
 8   destino_fecha     360 non-null    object
 9   fecha             360 non-null    object
 10  ndia              360 non-null    int64 
 11  wdia              360 non-null    int64 
 12  dhabil            360 non-null    int64 
 13  hora              360 non-null    int64 
 14  mins              360 non-null    int64 
 15  ddia              360 non-null    int64 
 16  trimestre         360 non-null    int64 
dtypes: int64

In [ ]:
#  ESTO AL DE DATA preparin

# insumos["feriados"]["df"] = pd.read_csv(
#     datadir + 'dias_festivos.csv'
# )
# derivados["feriados"]["df"]["nmes"] = derivados["feriados"]["df"]["nmes"].astype(str).apply(lambda x: x.zfill(2))
# derivados["feriados"]["df"]["fecha"] = derivados["feriados"]["df"]["anio"].astype(str) \
#     + '-' + derivados["feriados"]["df"]["nmes"].astype(str) \
#     + '-' + derivados["feriados"]["df"]["ndia"].astype(str)
# derivados["feriados"]["df"]["feriado"] = 1
# insumos["feriados"]["df"] = derivados["feriados"]["df"].sort_values('fecha').reset_index()
# derivados["feriados"]["df"] = derivados["feriados"]["df"].drop( 
#     columns=[ 'Unnamed: 0', 'anio', 'nmes', 'ndia', 'index' ]
# )
# # derivados["feriados"]["df"].to_csv(
# #   insumos["feriados"]["local"],
# #   index = False,
# # )
# derivados["feriados"]["df"]

In [18]:
insumos["feriados"]["df"].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   label    59 non-null     object
 1   type     59 non-null     object
 2   fecha    59 non-null     object
 3   feriado  59 non-null     int64 
dtypes: int64(1), object(3)
memory usage: 2.0+ KB


In [19]:
derivados["recorridos"]["df"] = pd.merge(
    left     = derivados["recorridos"]["df"],
    right    = insumos["feriados"]["df"],
    how      = 'left',
    left_on  = ['fecha'],
    right_on = ['fecha']
)
derivados["recorridos"]["df"] = derivados["recorridos"]["df"].drop( 
    columns=[ 'label','type' ]
)
derivados["recorridos"]["df"]["feriado"] = derivados["recorridos"]["df"]["feriado"].fillna(0).astype(int)
derivados["recorridos"]["df"]["dhabil"][ derivados["recorridos"]["df"]["feriado"] == 1 ] = 0
del insumos["feriados"]["df"]

In [20]:
# del insumos["feriados"]["df"]
# derivados["recorridos"]["df"][ derivados["recorridos"]["df"]["feriado"] == 1 ]

## FEATURE DISCOVERY

In [20]:
# DURACION EN MINUTOS 
derivados["recorridos"]["df"]["duracion_mins"] = 0
derivados["recorridos"]["df"].eval('duracion_mins = duracion / 60', inplace = True) # mins
derivados["recorridos"]["df"]["duracion_mins"] = derivados["recorridos"]["df"]["duracion_mins"].astype(int)
print(
    int(derivados["recorridos"]["df"]["duracion_mins"].mean()),
    derivados["recorridos"]["df"]["duracion_mins"].max()
)

22 53168


In [25]:
# # mean_dur = int(derivados["recorridos"]["df"]["duracion_mins"].mean())
# mean_dur = round( int(derivados["recorridos"]["df"]["duracion_mins"].mean()), -1 ) # 22 > 20
# bins = 20
# dur_cortes  = [ int( ( i * 0.25 ) * mean_dur ) for i in range( 0, bins, 1 )  ]
# labels = [ dur_cortes[ i ] for i in range( 0, len( dur_cortes ) -1, 1 ) ]
# labels= [0, 5, 11, 16, 22, 27, 33, 38, 44]
# derivados["recorridos"]["df"]["duracion_cat"] = pd.cut( 
#     derivados["recorridos"]["df"].duracion_mins,
# #     dur_cortes,
# #     labels = labels,
# #     right = True,
#     include_lowest = True,
# #     ordered = True,
#     bins = 4,
#     labels = False,
# )
# derivados["recorridos"]["df"]["duracion_cat"].unique()

array([0, 1, 2, 3])

In [ ]:
# # MINUTOS TO CATEGORIES
# labels = [ "{0}-{1}".format(i, i + top_dur) for i in range(0, top_dur, mean_dur) ]
# labels = [ "{0}-{1}".format(i, i + 15) for i in range(0, 265, 15) ]
# dur_etiquetas = [ 1, 2, 3, 4, 'M', 'ML', 'L', 'XL', 'XXL', 'OS' ]
# dur_cortes    = [ 0,    22.240011093532345,   15,  30,    60,     90, 120,  240, 320, 60000 ]
# derivados["recorridos"]["df"]["duracion_cat"] = pd.cut( 
#     derivados["recorridos"]["df"].duracion_mins,
# #     dur_cortes,
# #     labels = dur_etiquetas
#     right = False,
#     include_lowest = True,
#     bins = 10,
#     labels = False
# )
# derivados["recorridos"]["df"].unique()

### TIPO DE RECORRIDO: VUELTA / IDA

In [16]:
derivados["recorridos"]["df"]['vuelta'] = np.where( 
  derivados["recorridos"]["df"]['origen_id'] == derivados["recorridos"]["df"]['destino_id'], 1, 0
)
# derivados["recorridos"]["df"][ derivados["recorridos"]["df"]['vuelta'] == 1 ] 

### DISTANCIAS


In [ ]:
# !pip install openrouteservice

In [ ]:
derivados["estaciones"]["df"] = pd.read_csv(
 derivados["estaciones"]["local"],
#      ignore_index = True,
)
estaciones = [ x for x in derivados["estaciones"]["df"].set_index('id').sort_values('id').iterrows() ]
# del rutas
rutas = {}
nid = 0
for o in estaciones:
    origen_id = int(o[0]) # revisar
    print (origen_id)
    # https://ask.openrouteservice.org/t/could-not-find-point-within-a-radius-of-350-0-meters/2668/16
    origen_coords = ( o[1]['lon'], o[1]['lat'] ) # ORS takes lon,lat 
    for d in estaciones:
        destino_id = int(d[0]) # revisar
        destino_coords = ( d[1]['lon'], d[1]['lat'] ) # ibidem
        ruta_id = str( origen_id ) + 'to' + str( destino_id )
        vuelta_id = str( destino_id) + 'to' + str( origen_id )
        ruta = {
            "id" : ruta_id,
            "nid" : nid,
            "origen_id": origen_id,
            "destino_id": destino_id,
            "coords" : ( origen_coords, destino_coords ),
            "distancia" : 0,
            "duracion_estimada" : 0,
            "consultar" : ( origen_id != destino_id ),
            "complemetaria" : vuelta_id,
            "preexiste" : (ruta_id in rutas or vuelta_id in rutas),
        }
        nid += 1
        rutas[ruta_id] = ruta
print( 'rutas:', len( rutas ) )

rutas_calcular = {}
chunk = 0
cuenta_rutas = 0
for r in rutas:
    ruta = rutas[r]
    if ruta['consultar'] and not ruta["preexiste"]:
        if( (cuenta_rutas % 2000) == 0 ): # due to ors dayly cuota
            chunk = chunk + 1  
        ruta["chunk"] = chunk
        rutas_calcular[r] = ruta
        cuenta_rutas = cuenta_rutas + 1
print( 'rutas a calular:', len( rutas_calcular ) )

In [17]:
cuenta = 0
for r in rutas_calcular:
    ruta = rutas_calcular[r]
    if ruta["chunk"] == 3:
        cuenta += 1
print(cuenta)

2000


In [ ]:
from time import sleep
import openrouteservice as ors

ors_client = ors.Client(
    key = '5b3ce3597851110001cf624827a63bd0239f438caa1f7504352a9fc8'
) 
# del rutas_calculadas
# rutas_calculadas = pd.DataFrame()

# derivados["rutas"]["df"]  = pd.read_csv(
#   derivados["rutas"]["local"],
# )

# CARGAR RUTAS YA CALCULDAS
derivados["rutas"]["df"] = pd.read_csv(
  derivados["rutas"]["local"],
)
rutas_calculadas = derivados["rutas"]["df"] 
cuenta_calculadas = 1
for rc in rutas_calcular:
    ruta = rutas[rc]
    if ruta['chunk'] == 4: # ya hice el chunk 2.80
        if ( ( cuenta_calculadas % 40 ) == 0 ): # due to ors minute cuota
            print( 'durmiendo', cuenta_calculadas )
            print( 'rutas calculadas:', len( rutas_calculadas.index ) )
            rutas_calculadas
            sleep(60)
#         if 'directions' not in ruta:
        ruta['directions'] = ors_client.directions(
          ruta['coords'],
          profile = 'cycling-regular',
        )
        cuenta_calculadas = cuenta_calculadas + 1
        rutas_calculadas = rutas_calculadas.append(
           ruta,
           ignore_index = True
        )
print( 'rutas calculadas hoy:', len(rutas_calculadas.index))
# rutas_df = pd.concat(   derivados["rutas"]["df"] , rutas_calculadas ] )

rutas_calculadas.to_csv(
  derivados["rutas"]["local"],
  index = False,
)
# rutas_df.reset_index()

In [19]:
# print(len(rutas_calculadas.index))
rutas_calculadas.to_csv(
  derivados["rutas"]["local"],
  index = False,
)
rutas_calculadas

,chunk,complemetaria,consultar,coords,destino_id,directions,distancia,duracion_estimada,id,nid,origen_id,preexiste
0,1.0,1to0,1.0,"(('-58.3897856456985', '-34.6427424928089'), (...",1.0,"{'routes': [{'summary': {'distance': 7773.0, '...",0.0,0.0,0to1,1.0,0.0,0.0
1,1.0,2to0,1.0,"(('-58.3897856456985', '-34.6427424928089'), (...",2.0,"{'routes': [{'summary': {'distance': 7162.0, '...",0.0,0.0,0to2,2.0,0.0,0.0
2,1.0,3to0,1.0,"(('-58.3897856456985', '-34.6427424928089'), (...",3.0,"{'routes': [{'summary': {'distance': 5293.6, '...",0.0,0.0,0to3,3.0,0.0,0.0
3,1.0,4to0,1.0,"(('-58.3897856456985', '-34.6427424928089'), (...",4.0,"{'routes': [{'summary': {'distance': 6481.4, '...",0.0,0.0,0to4,4.0,0.0,0.0
4,1.0,5to0,1.0,"(('-58.3897856456985', '-34.6427424928089'), (...",5.0,"{'routes': [{'summary': {'distance': 8829.6, '...",0.0,0.0,0to5,5.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7995,4.0,99to50,1.0,"((-58.4010767000307, -34.5837350914111), (-58....",99.0,"{'routes': [{'summary': {'distance': 4377.3, '...",0.0,0.0,50to99,8775.0,50.0,0.0
7996,4.0,100to50,1.0,"((-58.4010767000307, -34.5837350914111), (-58....",100.0,"{'routes': [{'summary': {'distance': 4352.6, '...",0.0,0.0,50to100,8776.0,50.0,0.0
7997,4.0,101to50,1.0,"((-58.4010767000307, -34.5837350914111), (-58....",101.0,"{'routes': [{'summary': {'distance': 5291.3, '...",0.0,0.0,50to101,8777.0,50.0,0.0
7998,4.0,103to50,1.0,"((-58.4010767000307, -34.5837350914111), (-58....",103.0,"{'routes': [{'summary': {'distance': 1147.1, '...",0.0,0.0,50to103,8778.0,50.0,0.0


In [ ]:
rutas_df = pd.read_csv(
  derivados["rutas"]["local"],
)
# rutas_df
print( len( rutas_df.index ) )

In [ ]:
# OBTENER DISTANCIAS (DIRECTAS NO CALCULA CAMINO URBANO) GEOPY 
import geopy
distancias = pd.DataFrame()
estaciones = [ x for x in derivados["estaciones"]["df"].set_index('id').sort_values('id').iterrows() ]
for o in estaciones:
    print(cuenta_calls)
    origen_id = int( o[0] )
    origen_geocoords = ( o[1]['lat'], o[1]['lon'] )
    # distancias[origen_id] = {}
    for d in estaciones:
        destino_id = int( d[0] )
        destino_geocoords = ( d[1]['lat'], d[1]['lon'] ) 
        fila = {
          "id"  : str(origen_id) + 'to' + str(destino_id),
          "origen_id": origen_id,
          "destino_id": destino_id,
          "distancia" : 0,
        }
        if (
            destino_id != origen_id
        ): 
            geocoords = ( origen_coords, destino_coords )
            distancia = round( 
                geopy.distance.vincenty( origen_geocoords, destino_geocoords ).km, 
                2
            ) 
            fila["distancia"] = distancia
        distancias = distancias.append( fila, ignore_index = True )
distancias['origen_id'] = distancias['origen_id'].astype(int)
distancias.to_csv(
  datadir + 'distancias.csv',
  index = False,
)

distancias

### Distancias GEOPY

In [22]:
# esto queda en pararacion
# derivados["distancias"]["df"] = pd.read_csv(
#   derivados["distancias"]["local"]
# )
derivados["distancias"]["df"] = derivados["distancias"]["df"].rename(
    columns = { 'Unnamed: 0': 'id' }
).set_index('id')
# Hay que reshaping de matriz a lista 
distasncias_df = pd.DataFrame()
for f in derivados["distancias"]["df"].iterrows():
    oid = int( f[0] )
    fila = f[1]
    for col in derivados["distancias"]["df"].columns:
        did = int( col )
        distancia = fila[ col ]
        trayecto = {
            'origen_id'  : oid,
            'destino_id' : did,
            'distancia'  : distancia,
        }
        distasncias_df = distasncias_df.append(
            trayecto,
            ignore_index = True
        )

In [23]:
distasncias_df.isnull().sum()

destino_id    0
distancia     0
origen_id     0
dtype: int64

In [24]:
del derivados["distancias"]["df"]
distasncias_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   destino_id  40000 non-null  float64
 1   distancia   40000 non-null  float64
 2   origen_id   40000 non-null  float64
dtypes: float64(3)
memory usage: 937.6 KB


In [25]:
# derivados["recorridos"]["df"]["distancia"] = 0
derivados["recorridos"]["df"] = pd.merge(
    left     = derivados["recorridos"]["df"],
    right    = distasncias_df,
    how      = 'left', 
    left_on  = ['origen_id','destino_id'], 
    right_on = ['origen_id','destino_id']
)
del distasncias_df

In [26]:
derivados["recorridos"]["df"]["distancia"].mean()

2.1956260703548574

In [27]:
# derivados["recorridos"]["df"] = derivados["recorridos"]["df"].drop( columns = { 'duracion' } )
derivados["recorridos"]["df"]["distancia"] = derivados["recorridos"]["df"]["distancia"].fillna(0)
derivados["recorridos"]["df"].isnull().sum()

duracion            0
usuario_id          0
origen_id           0
origen_nombre       0
origen_fecha        0
origen_direccion    0
destino_id          0
destino_nombre      0
destino_fecha       0
fecha               0
ndia                0
wdia                0
dhabil              0
hora                0
mins                0
ddia                0
trimestre           0
feriado             0
duracion_mins       0
vuelta              0
distancia           0
dtype: int64

In [40]:
# mean_dis  =  derivados["recorridos"]["df"]["distancia"].mean()
# bins = 4
# dis_cortes  = [ int( ( i * 0.5 ) * mean_dis ) for i in range( 0, bins, 1 )  ]
# dis_labels = [ dis_cortes[ i ] for i in range( 0, len( dis_cortes ) -1, 1 ) ]
# derivados["recorridos"]["df"]["distancia_cat"] = pd.cut( 
#     derivados["recorridos"]["df"].distancia,
# #     dis_cortes,
# #     labels = dis_labels,
#     right = True,
#     include_lowest = True,
#     ordered = True,
#     bins = 4,
#     labels = False,
    
# )
# derivados["recorridos"]["df"]["distancia_cat"].unique()

array([0, 1, 2, 3])

In [ ]:
# derivados["recorridos"]["df"][ derivados["recorridos"]["df"]["vuelta"] == 0 ]

### VELOCIDAD APROXIMADA

In [28]:
derivados["recorridos"]["df"].eval(
    'velocidad_kmm = distancia / duracion_mins',
    inplace = True
)
derivados["recorridos"]["df"]["velocidad_kmm"].replace(
    np.inf,
    0,
    inplace = True
)
derivados["recorridos"]["df"]["velocidad_kmm"].mean()

0.12328109501919293

In [29]:
derivados["recorridos"]["df"]["velocidad_kmm"] = derivados["recorridos"]["df"]["velocidad_kmm"].fillna(0)
derivados["recorridos"]["df"].isnull().sum()

duracion            0
usuario_id          0
origen_id           0
origen_nombre       0
origen_fecha        0
origen_direccion    0
destino_id          0
destino_nombre      0
destino_fecha       0
fecha               0
ndia                0
wdia                0
dhabil              0
hora                0
mins                0
ddia                0
trimestre           0
feriado             0
duracion_mins       0
vuelta              0
distancia           0
velocidad_kmm       0
dtype: int64

In [42]:
# derivados["recorridos"]["df"]["velocidad_cat"] = pd.cut( 
#     derivados["recorridos"]["df"].velocidad_kmm,
#     bins = 4,
#     labels = False,
# #     right = True,
#     include_lowest = True,
# )
# derivados["recorridos"]["df"]["velocidad_cat"].unique()

array([ 0., nan,  1.,  2.,  3.])


### CLIMA
https://www.smn.gob.ar/descarga-de-datos
https://www.smn.gob.ar/datos-abiertos-smn


### Regular sequences: pd.date_range()

Regular date sequences 

In [43]:
# Lista de dias a obtener
primera_fecha  = derivados["recorridos"]["df"]["fecha"].min().split('-') 
sdate = date( int(primera_fecha[0]), int(primera_fecha[1]), int(primera_fecha[2]))
ultima_fecha  = derivados["recorridos"]["df"]["fecha"].max().split('-') 
fdate = date( int(ultima_fecha[0]), int(ultima_fecha[1]), int(ultima_fecha[2]))
dias = pd.date_range( sdate, fdate, freq = 'd' )

### Observaciones Horarias

```
$ curl -v --silent https://ssl.smn.gob.ar/dpd/descarga_opendata.php?file=observaciones/datohorario20210518.txt 2>&1 | grep AERO
!curl --silent https://ssl.smn.gob.ar/dpd/descarga_opendata.php?file=observaciones/datohorario20210518.txt | head -n 1
!curl --silent https://ssl.smn.gob.ar/dpd/descarga_opendata.php?file=observaciones/datohorario20210518.txt | grep AEROPARQUE

```

In [55]:
!curl --silent https://ssl.smn.gob.ar/dpd/descarga_opendata.php?file=observaciones/datohorario20210518.txt | head -n 10    

FECHA     HORA  TEMP   HUM   PNM    DD    FF     NOMBRE                                             
         [HOA]  [�C]   [%]  [hPa]  [gr] [km/hr]                                                     
18052021     0  11.2   83  1021.8    0    0     AEROPARQUE AERO                                     
18052021     1  10.8   82  1022.1    0    0     AEROPARQUE AERO                                     
18052021     2  11.4   77  1021.8  160    6     AEROPARQUE AERO                                     
18052021     3  10.8   82  1021.6  160    4     AEROPARQUE AERO                                     
18052021     4  10.2   86  1021.2  990    4     AEROPARQUE AERO                                     
18052021     5  10.2   86  1021.5  990    4     AEROPARQUE AERO                                     
18052021     6  10.2   84  1021.5  250    4     AEROPARQUE AERO                                     
18052021     7  10.1   81  1021.7  250    2     AEROPARQUE AERO                            

In [ ]:
## REVISAR FALTAN ESTOS DIAS
# ['2020-01-30', '2020-09-04', '2020-01-31', '2020-01-11',
#  '2020-10-21', '2020-02-13', '2020-10-14', '2020-09-19',
#  '2020-02-11', '2020-12-14', '2020-11-03', '2020-02-15',
#  '2020-10-19', '2020-09-02', '2021-02-14', '2021-01-28'],

""" Obtener datos horarios para cada dia  """
dh_formato = '%d%m%Y%H'
prefijo = 'https://ssl.smn.gob.ar/dpd/descarga_opendata.php?file=observaciones/datohorario'
sufijo = '.txt'
for i, d in enumerate(dias):
    dstr = d.strftime('%Y%m%d') 
    dato_horario = prefijo + dstr + sufijo
    req = urllib.request.Request( dato_horario )
    response = urllib.request.urlopen( req )
    lines = response.readlines()
    fecha = d.strftime( '%Y-%m-%d' )
    print( i, 'procesando', dato_horario )
    # encabezados = lines[0].decode("utf-8").split()  
    # ['FECHA', 'HORA', 'TEMP', 'HUM', 'PNM', 'DD', 'FF', 'NOMBRE']
    # [b'[HOA]', b'[\xbaC]', b'[%]', b'[hPa]', b'[gr]', b'[km/hr]']
    # fstl = lines[2].decode("utf-8").split()  
    # print( encabezados )
    # print( fstl )
    for l in lines:
        if b'AEROPARQUE' in l:
            row =  l.decode("utf-8").split()
            # dh = str(row[0]) + str(row[1])
            # dh_dt = pd.to_datetime(
            #   dh, 
            #   format = dh_formato
            # ) 
            dh_dic = {
                # 'DH_DT'   : dh_dt,
                'FECHA'   : fecha,
                'HORA'    : row[1],
                'TEMP'    : row[2], 
                'HUM'     : row[3], 
                'PNM'     : row[4], 
                'DD'      : row[5], 
                'FF'      : row[6], 
                # 'NOMBRE'  : row[7]
            }
            derivados["clima"]["df"] = derivados["clima"]["df"].append(
                dh_dic,
                ignore_index=True
            )
            
derivados["clima"]["df"]['DD'] = derivados["clima"]["df"]['DD'].replace('AEROPARQUE',170)
derivados["clima"]["df"]['FF'] = derivados["clima"]["df"]['FF'].replace( 'AERO', 13)
derivados["clima"]["df"]['FF'] = derivados["clima"]["df"]['FF'].replace( 'AEROPARQUE', 13)
derivados["clima"]["df"]['HORA'] = derivados["clima"]["df"]['HORA'].astype(str).astype(int)
derivados["clima"]["df"]['HUM'] = derivados["clima"]["df"]['HUM'].astype(float).astype(int).astype(int)
derivados["clima"]["df"]['DD'] = derivados["clima"]["df"]['DD'].astype(str).astype(int)
derivados["clima"]["df"]['FF'] = derivados["clima"]["df"]['FF'].astype(str).astype(int)
derivados["clima"]["df"]['TEMP'] = derivados["clima"]["df"]['TEMP'].astype(str).astype(float)
derivados["clima"]["df"]['PNM'] = derivados["clima"]["df"]['PNM'].astype(str).astype(float)
# derivados["clima"]["df"].isnull().sum()
derivados["clima"]["df"].to_csv(
    derivados["clima"]["local"],
)

In [ ]:
derivados["clima"]["df"].isnull().sum()
# derivados["clima"]["df"].to_csv(
#     derivados["clima"]["local"],
# )

In [63]:
# derivados["clima"]["df"].isnull().sum()
# # derivados["clima"]["df"].info()
derivados["clima"]["df"].to_csv(
    derivados["clima"]["local"],
)

#### Pronostico

```
!curl --silent https://ssl.smn.gob.ar/dpd/descarga_opendata.php?file=pron5d/pron20210101.txt
curl --silent https://ssl.smn.gob.ar/dpd/descarga_opendata.php?file=pron5d/pron20210101.txt | head -n 52
curl --silent https://ssl.smn.gob.ar/dpd/descarga_opendata.php?file=pron5d/pron20210101.txt | head -n 18
```

In [ ]:
!curl --silent https://ssl.smn.gob.ar/dpd/descarga_opendata.php?file=pron5d/pron20210101.txt

In [10]:
# AEROPARQUE 1 DIA 8 OBSERVACIONES c / 3 horas
!curl --silent https://ssl.smn.gob.ar/dpd/descarga_opendata.php?file=pron5d/pron20210101.txt | head -n 18 | tail -n 8 | awk '/1/ -F ":" {print $1,$2,$7}' 

01/ENE/2021 00Hs. 0.0
01/ENE/2021 03Hs. 0.0
01/ENE/2021 06Hs. 0.0
01/ENE/2021 09Hs. 0.0
01/ENE/2021 12Hs. 0.0
01/ENE/2021 15Hs. 0.0
01/ENE/2021 18Hs. 0.0
01/ENE/2021 21Hs. 0.0


In [ ]:
"""
Obtener Pronostico # AEROPARQUE 1 DIA 8 OBSERVACIONES c / 3 horas
upsamplig
"""

# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html

# https://machinelearningmastery.com/resample-interpolate-time-series-data-python/
# https://towardsdatascience.com/upsample-with-an-average-in-pandas-c029032c57ca

# from pandas import datetime
 
# def parser(x):
# 	return datetime.strptime('190'+x, '%Y-%m')
 
# series = read_csv('shampoo-sales.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
# upsampled = series.resample('D')
# print(upsampled.head(32))

dh_formato = '%d%m%Y%H'
prefijo = 'https://ssl.smn.gob.ar/dpd/descarga_opendata.php?file=pron5d/pron'
sufijo = '.txt'
for i, d in enumerate(dias):
    dstr = d.strftime('%Y%m%d') 
    dato_horario = prefijo + dstr + sufijo
    req = urllib.request.Request( dato_horario )
    response = urllib.request.urlopen( req )
    lines = response.readlines()
    fecha = d.strftime( '%Y-%m-%d' )
    print( i, 'procesando', dato_horario )
    # encabezados = lines[0].decode("utf-8").split()  
    # ['FECHA', 'HORA', 'TEMP', 'HUM', 'PNM', 'DD', 'FF', 'NOMBRE']
    # [b'[HOA]', b'[\xbaC]', b'[%]', b'[hPa]', b'[gr]', b'[km/hr]']
    # fstl = lines[2].decode("utf-8").split()  
    # print( encabezados )
    # print( fstl )
    for l in lines:
        if b'AEROPARQUE' in l:
            row =  l.decode("utf-8").split()
            # dh = str(row[0]) + str(row[1])
            # dh_dt = pd.to_datetime(
            #   dh, 
            #   format = dh_formato
            # ) 
            dh_dic = {
                # 'DH_DT'   : dh_dt,
                'FECHA'   : fecha,
                'HORA'    : row[1],
                'TEMP'    : row[2], 
                'HUM'     : row[3], 
                'PNM'     : row[4], 
                'DD'      : row[5], 
                'FF'      : row[6], 
                # 'NOMBRE'  : row[7]
            }
            derivados["clima"]["df"] = derivados["clima"]["df"].append(
                dh_dic,
                ignore_index=True
            )

In [31]:
# CARGAR CLIMA
derivados["clima"]["df"] = pd.read_csv(
  derivados["clima"]["local"],
).drop(columns = ['Unnamed: 0'])
# ver fechas y horas q esten todas
# ['2021-02-12', '2021-02-11']
derivados["clima"]["df"].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12411 entries, 0 to 12410
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DD      12411 non-null  int64  
 1   FECHA   12411 non-null  object 
 2   FF      12411 non-null  int64  
 3   HORA    12411 non-null  int64  
 4   HUM     12411 non-null  int64  
 5   PNM     12411 non-null  float64
 6   TEMP    12411 non-null  float64
dtypes: float64(2), int64(4), object(1)
memory usage: 678.9+ KB


In [32]:
derivados["clima"]["df"]['DD'] = derivados["clima"]["df"]['DD'].replace('AEROPARQUE',170)
derivados["clima"]["df"]['DD'] = derivados["clima"]["df"]['DD'].astype(str).astype(int)
derivados["clima"]["df"]['FF'] = derivados["clima"]["df"]['FF'].replace( 'AERO', 13)
derivados["clima"]["df"]['FF'] = derivados["clima"]["df"]['FF'].replace( 'AEROPARQUE', 13)
derivados["clima"]["df"]['FF'] = derivados["clima"]["df"]['FF'].astype(str).astype(int)

In [33]:
# formato = '%Y-%m-%d'
# derivados["clima"]["df"]['DATE'] = pd.to_datetime(
#   derivados["clima"]["df"]['DATE'], 
#   format = formato
# )
# # derivados["clima"]["df"]['TAVG'].unique()
# derivados["clima"]["df"]['TEMP'].unique()
# derivados["clima"]["df"] = derivados["clima"]["df"].rename(columns=[])
# derivados["clima"]["df"].info()

# derivados["clima"]["df"].info()
derivados["clima"]["df"].isnull().sum()

DD       0
FECHA    0
FF       0
HORA     0
HUM      0
PNM      0
TEMP     0
dtype: int64

In [34]:
# https://stackoverflow.com/a/56842359
# df3 = df1.drop('b', 1).merge(df2, 'left', on='a').set_index(df1.index)
# df1.combine_first(df3)
# df1.update(df1[['a', 'e']].merge(df2, 'left'))

derivados["recorridos"]["df"] = pd.merge(
    left     = derivados["recorridos"]["df"],
    right    = derivados["clima"]["df"],
    how      = 'left', 
    left_on  = ['fecha','hora'], 
    right_on = ['FECHA','HORA']
)
del derivados["clima"]["df"]
derivados["recorridos"]["df"].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3331632 entries, 0 to 3331631
Data columns (total 29 columns):
 #   Column            Dtype  
---  ------            -----  
 0   duracion          int64  
 1   usuario_id        int64  
 2   origen_id         int64  
 3   origen_nombre     object 
 4   origen_fecha      object 
 5   origen_direccion  object 
 6   destino_id        int64  
 7   destino_nombre    object 
 8   destino_fecha     object 
 9   fecha             object 
 10  ndia              int64  
 11  wdia              int64  
 12  dhabil            int64  
 13  hora              int64  
 14  mins              int64  
 15  ddia              int64  
 16  trimestre         int64  
 17  feriado           int64  
 18  duracion_mins     int64  
 19  vuelta            int64  
 20  distancia         float64
 21  velocidad_kmm     float64
 22  DD                float64
 23  FECHA             object 
 24  FF                float64
 25  HORA              float64
 26  HUM           

In [35]:
derivados["recorridos"]["df"].isnull().sum()

duracion                0
usuario_id              0
origen_id               0
origen_nombre           0
origen_fecha            0
origen_direccion        0
destino_id              0
destino_nombre          0
destino_fecha           0
fecha                   0
ndia                    0
wdia                    0
dhabil                  0
hora                    0
mins                    0
ddia                    0
trimestre               0
feriado                 0
duracion_mins           0
vuelta                  0
distancia               0
velocidad_kmm           0
DD                  14350
FECHA               14350
FF                  14350
HORA                14350
HUM                 14350
PNM                 14350
TEMP                14350
dtype: int64

In [36]:
insumos["clima"]["df"] = pd.read_csv(
    insumos["clima"]["local"]
)
insumos["clima"]["df"] = insumos["clima"]["df"].drop( columns = { 
    'STATION','NAME','LATITUDE','LONGITUDE','ELEVATION','TMAX','TMIN',
    'PRCP_ATTRIBUTES','TAVG_ATTRIBUTES','TMAX_ATTRIBUTES','TMIN_ATTRIBUTES'
})
insumos["clima"]["df"].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1613 entries, 0 to 1612
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DATE    1613 non-null   object 
 1   PRCP    356 non-null    float64
 2   TAVG    1613 non-null   float64
dtypes: float64(2), object(1)
memory usage: 37.9+ KB


In [37]:
insumos["clima"]["df"].describe()

,PRCP,TAVG
count,356.000000,1613.000000
mean,7.920225,18.632424
std,16.342953,5.202943
min,0.000000,5.900000
25%,0.000000,14.400000
50%,0.300000,18.900000
75%,7.900000,22.700000
max,112.000000,30.400000


In [ ]:
# REVISAR
# array(['2021-02-12', '2021-02-11'], dtype=object)

In [38]:
# REVISAR
insumos["clima"]["df"]['PRCP'] = insumos["clima"]["df"]['PRCP'].fillna(0)
insumos["clima"]["df"].isnull().sum()

DATE    0
PRCP    0
TAVG    0
dtype: int64

In [39]:
derivados["recorridos"]["df"] = pd.merge(
    left     = derivados["recorridos"]["df"],
    right    = insumos["clima"]["df"],
    how      = 'left', 
    left_on  = 'fecha', 
    right_on = 'DATE'
)
del insumos["clima"]["df"]
derivados["recorridos"]["df"].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3331632 entries, 0 to 3331631
Data columns (total 32 columns):
 #   Column            Dtype  
---  ------            -----  
 0   duracion          int64  
 1   usuario_id        int64  
 2   origen_id         int64  
 3   origen_nombre     object 
 4   origen_fecha      object 
 5   origen_direccion  object 
 6   destino_id        int64  
 7   destino_nombre    object 
 8   destino_fecha     object 
 9   fecha             object 
 10  ndia              int64  
 11  wdia              int64  
 12  dhabil            int64  
 13  hora              int64  
 14  mins              int64  
 15  ddia              int64  
 16  trimestre         int64  
 17  feriado           int64  
 18  duracion_mins     int64  
 19  vuelta            int64  
 20  distancia         float64
 21  velocidad_kmm     float64
 22  DD                float64
 23  FECHA             object 
 24  FF                float64
 25  HORA              float64
 26  HUM           

In [40]:
derivados["recorridos"]["df"] = derivados["recorridos"]["df"].drop( columns = { 'FECHA','HORA', 'DATE'} )
derivados["recorridos"]["df"].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3331632 entries, 0 to 3331631
Data columns (total 29 columns):
 #   Column            Dtype  
---  ------            -----  
 0   duracion          int64  
 1   usuario_id        int64  
 2   origen_id         int64  
 3   origen_nombre     object 
 4   origen_fecha      object 
 5   origen_direccion  object 
 6   destino_id        int64  
 7   destino_nombre    object 
 8   destino_fecha     object 
 9   fecha             object 
 10  ndia              int64  
 11  wdia              int64  
 12  dhabil            int64  
 13  hora              int64  
 14  mins              int64  
 15  ddia              int64  
 16  trimestre         int64  
 17  feriado           int64  
 18  duracion_mins     int64  
 19  vuelta            int64  
 20  distancia         float64
 21  velocidad_kmm     float64
 22  DD                float64
 23  FF                float64
 24  HUM               float64
 25  PNM               float64
 26  TEMP          

In [41]:
derivados["recorridos"]["df"].isnull().sum()

duracion                0
usuario_id              0
origen_id               0
origen_nombre           0
origen_fecha            0
origen_direccion        0
destino_id              0
destino_nombre          0
destino_fecha           0
fecha                   0
ndia                    0
wdia                    0
dhabil                  0
hora                    0
mins                    0
ddia                    0
trimestre               0
feriado                 0
duracion_mins           0
vuelta                  0
distancia               0
velocidad_kmm           0
DD                  14350
FF                  14350
HUM                 14350
PNM                 14350
TEMP                14350
PRCP                21005
TAVG                21005
dtype: int64

In [44]:
# FILL NA CON PROMEDIO
for c in derivados["recorridos"]["df"].columns:
    nulls = derivados["recorridos"]["df"][c].isnull().sum()
    if nulls:
        mean = derivados["recorridos"]["df"][c].mean()
        fill = mean # or 0
        types = derivados["recorridos"]["df"][c].dtypes
        print( c, nulls, types, fill)
        derivados["recorridos"]["df"][c].fillna( mean, inplace = True)
# derivados["recorridos"]["df"]

DD 14350 float64 156.64104709819665
FF 14350 float64 14.931178295966397
HUM 14350 float64 64.6412668564204
PNM 14350 float64 1015.1488488165911
TEMP 14350 float64 20.561729029970913
PRCP 2885453 float64 6.71009191378348
TAVG 21005 float64 19.582630359747565
TMAX 1623205 float64 24.54931805690263
TMIN 459725 float64 15.440816467942744


In [45]:
derivados["recorridos"]["df"].isnull().sum()

duracion            0
usuario_id          0
origen_id           0
origen_nombre       0
origen_fecha        0
origen_direccion    0
destino_id          0
destino_nombre      0
destino_fecha       0
fecha               0
ndia                0
wdia                0
dhabil              0
hora                0
mins                0
ddia                0
trimestre           0
feriado             0
duracion_mins       0
vuelta              0
distancia           0
velocidad_kmm       0
DD                  0
FF                  0
HUM                 0
PNM                 0
TEMP                0
PRCP                0
TAVG                0
TMAX                0
TMIN                0
dtype: int64

In [42]:
# recorridos_clima 
# derivados["recorridos"]["df"].to_csv(
#   derivados["recorridos"]["local"],
#   index = False,
# )

compression_options = dict(
    method ='zip',
    archive_name = 'recorridos_clima_2020-2021.csv'
)
derivados["recorridos"]["df"].to_csv(
    datadir + 'recorridos.zip', 
    compression = compression_options,
    index = False,

)

In [79]:
derivados["recorridos"]["df"].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3331632 entries, 0 to 3331631
Data columns (total 34 columns):
 #   Column            Dtype  
---  ------            -----  
 0   duracion          int64  
 1   usuario_id        int64  
 2   origen_id         int64  
 3   origen_nombre     object 
 4   origen_fecha      object 
 5   origen_direccion  object 
 6   destino_id        int64  
 7   destino_nombre    object 
 8   destino_fecha     object 
 9   fecha             object 
 10  ndia              int64  
 11  wdia              int64  
 12  dhabil            int64  
 13  hora              int64  
 14  mins              int64  
 15  ddia              int64  
 16  trimestre         int64  
 17  feriado           int64  
 18  duracion_mins     int64  
 19  duracion_cat      int64  
 20  vuelta            int64  
 21  distancia         float64
 22  distancia_cat     int64  
 23  velocidad_kmm     float64
 24  velocidad_cat     float64
 25  DD                float64
 26  FF            

In [ ]:
# GRUPING USERS
usurios = derivados["recorridos"]["df"].groupby(
    ['usuario_id']
).agg(
  salidas = ( 'destino_id',  'count'),
  vueltas = ( 'vuelta',  'sum'),
  velocidad = ( 'velocidad_kmm',  'max'),
  distancia = ( 'distancia',  'max')
)
# usurios = usurios.pivot_table(
#     index   = [ 'usuario_id'],
#     columns = "salidas",
#     values  = "salidas",
#     # aggfunc = 'count',
# ).sort_values('origen_id')
# demanda_diaria # comprobar

In [ ]:
usurios = usurios[ ~usurios.isin([np.nan, np.inf, -np.inf]).any(1) ]
# usurios
usurios.sort_values( 'velocidad' , ascending = False).head(30)

# Analisis Exploratorio de Datos Espaciales
(Exploratory Spatial Data Analysis ESDA)

https://alcidanalytics.com/p/geographic-heatmap-in-python

https://alysivji.github.io/getting-started-with-folium.html

https://medium.com/analytics-vidhya/measure-driving-distance-time-and-plot-routes-between-two-geographical-locations-using-python-39995dfea7e

In [17]:
demanda_viz = derivados["recorridos"]["df"]
# Fitro x dia
# DIA DE MAS DEMANDA 2021-03-05 14061
fecha_viz   = '2020-03-05'
demanda_viz = demanda_viz[ demanda_viz["fecha"] == fecha_viz ]
demanda_viz = demanda_viz.groupby(
    ['origen_nombre', 'origen_id', 'hora', 'origen_lon','origen_lat']
).agg(
  salidas = ( 'hora',  'count' ),
  nombre  = ( 'origen_nombre',  'first' ),
  id      = ( 'origen_id',  'first' ),
  lon     = ( 'origen_lon',  'first' ),
  lat     = ( 'origen_lat',  'first' ),
)
# ).reset_index()
# df.set_index(['id', 'date', 'location'], append=True)
demanda_viz = demanda_viz.pivot_table(
    index = ['nombre',  'id', 'lon','lat'],
    columns = "hora",
    values = "salidas",
    fill_value = 0,
    # aggfunc = 'sum',
).sort_values('id')

In [18]:
#... where m is my map object. And 5 is the time (seconds) to render the map.
# https://stackoverflow.com/questions/53565979/export-a-folium-map-as-a-png
# https://blog.ouseful.info/2015/12/15/grabbing-screenshots-of-folium-produced-choropleth-leaflet-maps-using-selenium/
# from time import sleep

import io
from PIL import Image
import folium 
from folium.features import DivIcon
import selenium

import math
def mapear(x): 
    return int( (x / 10 ) * 255)

In [19]:
# del m
m = ''
for h in demanda_viz.columns.to_list():
    del m
    m = folium.Map(
#         [ -34.606499,-58.381094 ], # obelisco
        [-34.606739, -58.435539 ], # parque centenario
        zoom_start = 13,
        zoom_control=False,
        scrollWheelZoom=False,
        dragging=False
#         tiles='stamentoner'
    )
    folium.map.Marker(
        [ -34.559920, -58.372211 ],
        icon = DivIcon(
            icon_size=(600,400),
            icon_anchor=(0,0),
            html='<div style="font-size: 20pt;font-family: monospace;">' \
            + fecha_viz + ' HORA:' + "{:02d}".format( h ) \
            + '</div>',
        )
    ).add_to(m)
    demandas = demanda_viz[h]
    estaciones = demanda_viz[h].reset_index()
    for e in estaciones.iterrows():
        estacion = e[1]
        folium.CircleMarker(
            [ estacion['lat'], estacion['lon'] ],
            radius = estacion[h] * 5, 
            popup = "{0}\:{1}\demanda:{2}".format(
                estacion['id'],
                estacion['nombre'],
                str(estacion[h]),
            ),
            fill_color = "#{0:02x}{1:02x}{2:02x}".format(
                mapear( int(estacion[h]) ),
                255 - mapear( int(estacion[h]) ) ,
                255 - mapear( int(estacion[h]) )
            )
        ).add_to(m)
#     m.save( 'ESDA/' + fecha_viz + '_' + "{:02d}".format( h ) + '_' + 'map.html')
    img_data = m._to_png(5)
    img = Image.open(io.BytesIO(img_data))
    img.save( 'ESDA/' + fecha_viz + '_' + "{:02d}".format( h ) + '_' + 'map.png' )

In [ ]:
# !convert -density 72 'ESDA/2020-03-01_00_map.html' '2020-03-01_00_map.html'


In [20]:
# !ffmpeg -pattern_type glob -i './ESDA/*.png' -vf zoompan=z=1:d=4:s=1366x683:fps=5,framerate=25 -vcodec libx264 -acodec aac ESDA.mp4
!ffmpeg -pattern_type glob -i './ESDA/*.png' -vf zoompan=z=1:d=4:s=1366x683:fps=5,framerate=25 -vcodec libx264 -an ESDA.mp4

ffmpeg version n4.4 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11.1.0 (GCC)
  configuration: --prefix=/usr --disable-debug --disable-static --disable-stripping --enable-amf --enable-avisynth --enable-cuda-llvm --enable-lto --enable-fontconfig --enable-gmp --enable-gnutls --enable-gpl --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libdav1d --enable-libdrm --enable-libfreetype --enable-libfribidi --enable-libgsm --enable-libiec61883 --enable-libjack --enable-libmfx --enable-libmodplug --enable-libmp3lame --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libopenjpeg --enable-libopus --enable-libpulse --enable-librav1e --enable-librsvg --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtheora --enable-libv4l2 --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxcb --enable-libxml2 --enable-libxvid --e